In [1]:
%%capture
# run this cell if this package is not installed
# !pip install nbformat

In [2]:
%%capture
%run setup.ipynb

In [3]:
def normalize_document(doc):
    wpt = nltk.WordPunctTokenizer()
    stop_words = set(nltk.corpus.stopwords.words('english'))

    # sentences to be removed
    p1 = "Chat Conversation Start"
    p2 = "Chat Conversation End"
    
    # Remove p1 only the first time it appears
    doc = re.sub(rf'^\s*{re.escape(p1)}\s*', '', doc)

    # Remove p2 only the last time it appears
    doc = re.sub(rf'\s*{re.escape(p2)}\s*$', '', doc)
    
    # remove email addresses
    doc = re.sub(r'\b\S*@\S*\.\S*\b', '', doc)
    # lowercase and remove special characters\whitespace
    doc = re.sub(r'[^a-zA-Z\s]', '', str(doc), re.I | re.A)
    doc = doc.lower()
    doc = doc.strip()
            
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    
    return doc


In [4]:
df_norm = df['Content'].apply(normalize_document)

In [5]:
df_norm.iloc[0]

'u good girls bed haha omg miss lot lovely friend miss smile long time yah u yes crazy crzy friend hehe im friend house u great came home well want ask everything good heard happened cat judi im really sad hope good spent good time u everything great dont worry fine new news u im excited since im going usa monday see friend u fantastic city u new jersey lot work u long u one week ill back meet u good im happy try safe enjoy lot maybe plan travel later friends still remember traveled last time montreal really fun good time hope repeat soon spend crazy time like soon'

In [6]:
# %%capture
# # spacy.cli.download("en_core_web_sm")

# nlp = spacy.load("en_core_web_sm")

In [7]:
# def lemmatizer(text):        
#     sent = []
#     doc = nlp(text)
#     #print(doc)
#     for word in doc:
#         sent.append(word.lemma_)
#     return " ".join(sent)

In [8]:
cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(df_norm)
cv_matrix

<11336x119336 sparse matrix of type '<class 'numpy.int64'>'
	with 1252012 stored elements in Compressed Sparse Row format>

In [9]:
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [10]:
# get all unique words in the corpus
vocab = cv.get_feature_names_out()
# show document feature vectors
pd.DataFrame(cv_matrix, columns=vocab)

,00,000,0000,000000,00000000,00000000b,00000001,00000001b,00000010,00000010b,...,zzo,zzr,zzt,zzy_3w,zzzoh,zzzs,zzzz,zzzzzz,zzzzzzt,ãªtre
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11331,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11332,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11333,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11334,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
from sklearn.feature_extraction.text import TfidfTransformer

del df, df_norm

tt = TfidfTransformer(norm='l2', use_idf=True)
tt_matrix = tt.fit_transform(cv_matrix)

del cv_matrix

tt_matrix = tt_matrix.toarray()
vocab = cv.get_feature_names_out()
pd.DataFrame(np.round(tt_matrix, 2), columns=vocab)

del tt_matrix